In [2]:
import pandas as pd
import numpy as np
import re
import os

def process_txt(file_name):
    file = open(file_name, 'r', encoding="utf-8")
    lines = file.readlines()
    file.close()
    
    lines = [line[:-1] for line in lines] #remove WEBVTT
    df = pd.DataFrame(data=lines)
    
    #resolving diff between xlsx and txt format of Zoom transcript
    df.drop([0,1], inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.drop(range(3,len(df),4), inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df

def reformat(file_name, save_ext, header=False):
    print(file_name)
    base_name, extension = os.path.splitext(file_name)

    if extension == ".xlsx":
        xl = pd.ExcelFile(file_name)
        df = xl.parse("Sheet1", index_col=None, header=None)
    if extension == ".txt":
        df = process_txt(file_name)

    df.columns = ['raw']

    #drop number lines
    df.drop(range(0,len(df),3), inplace=True)
    df.reset_index(inplace=True, drop=True)

    df_timestamp = df.iloc[0::2, :]
    df_timestamp.reset_index(inplace=True, drop=True)

    df_text = df.iloc[1::2, :]
    df_text.reset_index(inplace=True, drop=True)

    df = pd.concat([df_timestamp,df_text],axis=1)
    df.columns = ['timestamp','text']

    participant_names = []
    for index,row in df.iterrows():
        #regex pattern for name, in form of "NAME:"
        p = re.compile("^(.+):", re.I)
        name = p.search(row["text"])
        text = p.sub("",row["text"])
        df.loc[index]['text'] = text
        if name:
            participant_names.append(name.group()[:-1])
        else:
            participant_names.append("unclear")
        p2 = re.compile("\d\d:\d\d:\d\d.\d\d(?=\d)")
        inq_time = p2.search(row["timestamp"])
        df.loc[index]['timestamp'] = "["+inq_time.group()+"]"
    df["name"]=participant_names
    df = df[["timestamp","name","text"]]
    
    save_file_name = base_name+"_cleaned"+save_ext
    if save_ext==".xlsx":
        df.to_excel(save_file_name, index=False, header=header)
    if save_ext==".txt":
        df.to_csv(save_file_name, sep=';', index=False, header=header)

directory = "./VSP"
for filename in os.listdir(directory):
    if not filename.endswith("_cleaned.xlsx"):
        reformat(directory+"./"+filename, ".xlsx", header=True)
# reformat("200430_SH_Period 1 Bio Interview.txt", ".txt")

./VSP./VSP Day 1.txt
./VSP./VSP Day 2.txt
./VSP./VSP Day 3.txt
./VSP./VSP Day 4.txt
./VSP./VSP Day 5.txt
./VSP./VSP Day 6.txt
./VSP./VSP Day 7.txt
./VSP./VSP Day 8.txt
